# Model Merging 

I am now at the end of the pipeline that I defined in `N04_radio_info.ipynb`:

$Radio Message → [Sentiment Analysis] → [Intent Classification] → [Entity Extraction] → Structured Data$

The Structured data is neccesary to output a **standarized JSON format** that would be used by the logical agent in week 5 planning, alongside other models.

## Objetives of this notebook

The objectives and task to be done in this notebook are the following ones:

1. *Configure the mmodel paths*: first step, where I will configure the structure of the JSON.

2. *Load the pretrained models*: I will implement a function that will load the three elected models for making the radio analysis.

3. *Prediction functions*: for each mdel, it is necessary to implement a function where each models make their predictions.

4. *Audio Transcription*: first example usage, where I´ll use again Whisper for transcribing the radio message. Important for the urute if we want to directly pass the agent radio messages by radio.

5. *Integrating the pipeline into a single function*: this function will make the predictions of the three models.

6. *Analyzing text or audio examples*: two cells are going to be implemented for making the predictions in different types of radio formats.


---

## 1. Import Necessary Libraries

In [2]:
import json
import os
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import spacy
import whisper

## 2. Configuration

In this cell, I will configure the format of the JSON. It is necessary to specify:

- The model paths: 
    - ``../../outputs/week4/models/best_roberta_sentiment_model.pt``: best sentiment model developed.
    - ``../../outputs/week4/models/best_roberta_large_intent_model.pt``: best intent model developed.
    - ``../../outputs/week4/models/best_focused_bert_model.pt``: best NER model.
    - `Whisper turbo` model.
    
- Label mapppings of the models:
    - **sentiment**: positive, negative, neutral.
    - **intention**: INFORMATION, PROBLEM, ORDER, STRATEGY, WARNING, QUESTION.
    - **NER**: ACTION, SITUATION, INCIDENT, STRATEGY_INSTRUCTION, POSITION_CHANGE, PIT_CALL, TRACK_CONDITION, TECHNICAL_ISSUE, WEATHER.

- Demo input for making the correct structure.

In [6]:
# Configure model paths and settings - adjusted to match your specific models
CONFIG = {
    # Model paths
    "sentiment_model_path": "../../outputs/week4/models/best_roberta_sentiment_model.pt",
    "intent_model_path": "../../outputs/week4/models/best_roberta_large_intent_model.pt",
    "ner_model_path": "../../outputs/week4/models/best_focused_bert_model.pt",
    "whisper_model_size": "turbo",  # Using Whisper turbo 
    
    # Label mappings
    "sentiment_labels": ["positive", "negative", "neutral"],
    "intent_labels": ["INFORMATION", "PROBLEM", "ORDER", "STRATEGY", "WARNING", "QUESTION"],
    
    # Entity mapping (SpaCy entity labels → output format)
    "entity_mapping": {
        "ACTION": "action",
        "SITUATION": "situation",
        "INCIDENT": "incident",
        "STRATEGY_INSTRUCTION": "strategy",
        "POSITION_CHANGE": "position",
        "PIT_CALL": "pit",
        "TRACK_CONDITION": "track",
        "TECHNICAL_ISSUE": "technical",
        "WEATHER": "weather"
    },
    
    # Demo inputs
    "example_text": "Box this lap for softs, Hamilton is catching up"
}

print("Configuration loaded successfully")


Configuration loaded successfully
